In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
# support function

def create_histogram(data, x: str, bins: int, kde: bool):
    sns.histplot(data=data, x=x, bins=bins, kde=kde)
    plt.title(f"{x} Distribution")
    plt.xlabel(x)
    plt.ylabel("Frequency")
    plt.show()

def create_scatter(data, x, y, bins: int, kde: bool, title: str=""):
    sns.scatterplot(data=data, x=x, y=y, bins=bins, kde=kde)
    plt.title(title)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()

In [3]:
melbourne_path = "..\\..\\Kaggle\\data\\melb_data.csv"
melbourne_data = pd.read_csv(melbourne_path, index_col=0)
melbourne_data.head()

,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
Suburb,,,,,,,,,,,,,,,,,,,,
Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [4]:
melbourne_data.columns

Index(['Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG', 'Date',
       'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude',
       'Regionname', 'Propertycount'],
      dtype='object')

In [5]:
melbourne_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13580 entries, Abbotsford to Yarraville
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Address        13580 non-null  object 
 1   Rooms          13580 non-null  int64  
 2   Type           13580 non-null  object 
 3   Price          13580 non-null  float64
 4   Method         13580 non-null  object 
 5   SellerG        13580 non-null  object 
 6   Date           13580 non-null  object 
 7   Distance       13580 non-null  float64
 8   Postcode       13580 non-null  float64
 9   Bedroom2       13580 non-null  float64
 10  Bathroom       13580 non-null  float64
 11  Car            13518 non-null  float64
 12  Landsize       13580 non-null  float64
 13  BuildingArea   7130 non-null   float64
 14  YearBuilt      8205 non-null   float64
 15  CouncilArea    12211 non-null  object 
 16  Lattitude      13580 non-null  float64
 17  Longtitude     13580 non-null  float64
 1

In [6]:
melbourne_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Rooms,13580.0,2.937997e+00,0.955748,1.00000,2.000000,3.000000,3.000000e+00,1.000000e+01
Price,13580.0,1.075684e+06,639310.724296,85000.00000,650000.000000,903000.000000,1.330000e+06,9.000000e+06
Distance,13580.0,1.013778e+01,5.868725,0.00000,6.100000,9.200000,1.300000e+01,4.810000e+01
Postcode,13580.0,3.105302e+03,90.676964,3000.00000,3044.000000,3084.000000,3.148000e+03,3.977000e+03
Bedroom2,13580.0,2.914728e+00,0.965921,0.00000,2.000000,3.000000,3.000000e+00,2.000000e+01
Bathroom,13580.0,1.534242e+00,0.691712,0.00000,1.000000,1.000000,2.000000e+00,8.000000e+00
Car,13518.0,1.610075e+00,0.962634,0.00000,1.000000,2.000000,2.000000e+00,1.000000e+01
Landsize,13580.0,5.584161e+02,3990.669241,0.00000,177.000000,440.000000,6.510000e+02,4.330140e+05
BuildingArea,7130.0,1.519676e+02,541.014538,0.00000,93.000000,126.000000,1.740000e+02,4.451500e+04
YearBuilt,8205.0,1.964684e+03,37.273762,1196.00000,1940.000000,1970.000000,1.999000e+03,2.018000e+03


In [7]:
melbourne_data.isna().sum()

Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64

In [8]:
# simple missing value handling
melbourne_data_clean = melbourne_data.dropna(axis=0)
melbourne_data_clean.isna().sum()

Address          0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
BuildingArea     0
YearBuilt        0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
dtype: int64

In [9]:
# y = melbourne_data_clean.Price
# numeric_feature = melbourne_data_clean.select_dtypes(include='number')
# selected_feature = numeric_feature.columns
# print(selected_feature)
# selected_feature = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude']
# X = melbourne_data_clean[selected_feature]

y = melbourne_data_clean.Price
X = melbourne_data_clean.select_dtypes(include='number')
X = X.drop(columns=['Price'], axis=1)

# print(f"Selected feature {X.columns}")

In [10]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [11]:
model = DecisionTreeRegressor()
model.fit(train_X, train_y)

val_predict = model.predict(val_X)
print(mean_absolute_error(val_y, val_predict))

235183.00581020012


In [12]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    val_predict = model.predict(val_X)
    mae = mean_absolute_error(val_y, val_predict)
    return(mae)

for max_leaf_nodes in range(600, 610):
    mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print(f"{max_leaf_nodes} \t\t {mae}")


600 		 227091.6021328713
601 		 226928.74089412147
602 		 226928.74089412147
603 		 226725.48259101674
604 		 226760.81260423575
605 		 226760.81260423575
606 		 226774.67684844858
607 		 226704.1244183093
608 		 227089.58054384938
609 		 227250.97499188036


In [15]:
model = DecisionTreeRegressor(random_state=1)
model.fit(train_X, train_y)
val_predict = model.predict(val_X)
print(mean_absolute_error(val_y, val_predict))

233220.79793415105
